In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
file_path = '/content/drive/MyDrive/datasets/Delivery truck trip data.xlsx'

df = pd.read_excel(file_path)

gdf = gpd.GeoDataFrame(df)

<ipython-input-4-fd9ecd7511ca>:8: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel(file_path)


In [ ]:
gdf.head()

,GpsProvider,BookingID,Market/Regular,BookingID_Date,vehicle_no,Origin_Location,Destination_Location,Org_lat_lon,Des_lat_lon,Data_Ping_time,...,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,Minimum_kms_to_be_covered_in_a_day,Driver_Name,Driver_MobileNo,customerID,customerNameCode,supplierID,supplierNameCode,Material Shipped
0,CONSENT TRACK,MVCV0000927/082021,Market,2020-08-17 14:59:01.000,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",2020-08-24 00:05:09,...,320.0,NaN,NaN,NaN,NaN,ALLEXCHE45,Ashok leyland limited,VIJEXHOSR7,VIJAY TRANSPORT,BRACKET / GRAB HANDLE
1,VAMOSYS,VCV00014271/082021,Regular,2020-08-27 16:22:22.827,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:28,...,103.0,NaN,NaN,RAMESH,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,VJLEXSHE09,VJ LOGISTICS,ZB MODEL PLATE / 3143
2,CONSENT TRACK,VCV00014382/082021,Regular,2020-08-27 17:59:24.987,TN22AR2748,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8710,79.7390","11.8710,79.7390",2020-08-28 09:05:09,...,300.0,NaN,NaN,GIRI,NaN,LUTGCCHE06,Lucas tvs ltd,GSTEXLAK1Q,G.S. TRANSPORT,LETTERING / FUSO
3,VAMOSYS,VCV00014743/082021,Regular,2020-08-28 00:48:24.503,TN28AQ0781,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:31,...,61.0,NaN,NaN,RAVI,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,ARVEXNAM09,ARVINTH TRANSPORT,LU STRUT RA / RADIUS ROD
4,VAMOSYS,VCV00014744/082021,Regular,2020-08-28 01:23:19.243,TN68F1722,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8720,79.6320","11.8720,79.6320",2020-08-28 12:40:29,...,240.0,NaN,NaN,TAMIL,NaN,LUTGCCHE06,Lucas tvs ltd,SRTEXKOR96,SR TRANSPORTS,WISHBONE / V ROD/HDT


In [ ]:
gdf.Org_lat_lon.value_counts()

16.560192249175344,80.792293091599547    1189
18.750621,73.87719                        428
12.0001,79.74839949999999                 409
26.192290403509681,91.751276775362513     380
12.8390,79.9540                           342
                                         ... 
17.248088,78.395599                         1
28.3540,76.9390                             1
19.000914,73.20576                          1
17.464871,82.929988                         1
12.0001,79.7483995                          1
Name: Org_lat_lon, Length: 173, dtype: int64

In [ ]:
from shapely.geometry import Point


gdf['Org_lat_lon_geometry'] = gdf['Org_lat_lon'].apply(lambda x: Point(float(x.split(',')[1]), float(x.split(',')[0])))

gdf['Des_lat_lon_geometry'] = gdf['Des_lat_lon'].apply(lambda x: Point(float(x.split(',')[1]), float(x.split(',')[0])))

gdf = gdf.set_geometry("Org_lat_lon_geometry")
gdf = gdf.set_geometry("Des_lat_lon_geometry")


gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 34 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   GpsProvider                         5927 non-null   object        
 1   BookingID                           6880 non-null   object        
 2   Market/Regular                      6880 non-null   object        
 3   BookingID_Date                      6880 non-null   datetime64[ns]
 4   vehicle_no                          6880 non-null   object        
 5   Origin_Location                     6880 non-null   object        
 6   Destination_Location                6880 non-null   object        
 7   Org_lat_lon                         6880 non-null   object        
 8   Des_lat_lon                         6880 non-null   object        
 9   Data_Ping_time                      5927 non-null   datetime64[ns]
 10  Planned_ETA     

In [ ]:
india = gpd.read_file("/content/drive/MyDrive/datasets/india_district.geojson")

In [ ]:

G = nx.DiGraph()


coord_to_node = {}


for index, row in gdf.iterrows():

    org_coord = row['Org_lat_lon_geometry']
    des_coord = row['Des_lat_lon_geometry']


    if org_coord not in coord_to_node:
        coord_to_node[org_coord] = len(coord_to_node)
        G.add_node(len(coord_to_node) - 1, pos=org_coord, color='red')
    if des_coord not in coord_to_node:
        coord_to_node[des_coord] = len(coord_to_node)
        G.add_node(len(coord_to_node) - 1, pos=des_coord, color='orange')


    G.add_edge(coord_to_node[org_coord], coord_to_node[des_coord])



node_colors = [data['color'] for _, data in G.nodes(data=True)]

degrees = dict(G.degree())


node_sizes = [deg * 10 for _, deg in degrees.items()]

pos = {node: (data['pos'].x, data['pos'].y) for node, data in G.nodes(data=True)}
plt.figure(figsize=(30, 30))


india.plot(ax=plt.gca(), color='lightgrey', edgecolor='black')


nx.draw(G, pos, with_labels=False, node_size=node_sizes, node_color=node_colors, edge_color='b', alpha=0.5)

plt.title('Перевозки в Индии')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
average_degree = sum(dict(G.degree()).values()) / len(G)
print("Средняя степень вершины:", average_degree)

# average_shortest_path_length = nx.average_shortest_path_length(G)
# print("Средний минимальный путь:", average_shortest_path_length)

# diameter = nx.diameter(G)
# print("Диаметр графа:", diameter)


Средняя степень вершины: 2.858974358974359
